In [59]:
import re
from operator import itemgetter
from collections import Counter
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
from random import sample
from string import digits
from sklearn import preprocessing
import numpy as np

In [2]:
file_path = r"C:\Users\Samuel\Documents\python_class_question.html" 

In [3]:
with open(file_path, "r") as file_obj:
    html_txt  = file_obj.readlines()

In [4]:
html_txt[40]

'\t\t<td>MONDAY</td>\n'

In [5]:
days_pattern = r"\s\s<td>(\w+)</td>\s"

days = []

for i in html_txt:
    match_name = re.findall(days_pattern, i)
    
    if len(match_name) != 0:
        days.append(match_name)

days = [j for i in days for j in i]

In [6]:
days

['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY']

In [7]:
html_txt[41]

'\t\t<td>GREEN, YELLOW, GREEN, BROWN, BLUE, PINK, BLUE, YELLOW, ORANGE, CREAM, ORANGE, RED, WHITE, BLUE, WHITE, BLUE, BLUE, BLUE, GREEN</td>\n'

In [8]:
colors_pattern = r"\s\s<td>(\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+,\s\w+)</td>\s"

color = []

for i in html_txt:
    match_name = re.findall(colors_pattern, i)
    
    if len(match_name) != 0:
        color.append(match_name)

colors = [j.split(", ") for i in color for j in i] 

In [9]:
colors[0]

['GREEN',
 'YELLOW',
 'GREEN',
 'BROWN',
 'BLUE',
 'PINK',
 'BLUE',
 'YELLOW',
 'ORANGE',
 'CREAM',
 'ORANGE',
 'RED',
 'WHITE',
 'BLUE',
 'WHITE',
 'BLUE',
 'BLUE',
 'BLUE',
 'GREEN']

In [10]:
data = {i:j for i,j in zip(days, colors)} 

In [11]:
data.items()

dict_items([('MONDAY', ['GREEN', 'YELLOW', 'GREEN', 'BROWN', 'BLUE', 'PINK', 'BLUE', 'YELLOW', 'ORANGE', 'CREAM', 'ORANGE', 'RED', 'WHITE', 'BLUE', 'WHITE', 'BLUE', 'BLUE', 'BLUE', 'GREEN']), ('TUESDAY', ['ARSH', 'BROWN', 'GREEN', 'BROWN', 'BLUE', 'BLUE', 'BLEW', 'PINK', 'PINK', 'ORANGE', 'ORANGE', 'RED', 'WHITE', 'BLUE', 'WHITE', 'WHITE', 'BLUE', 'BLUE', 'BLUE']), ('WEDNESDAY', ['GREEN', 'YELLOW', 'GREEN', 'BROWN', 'BLUE', 'PINK', 'RED', 'YELLOW', 'ORANGE', 'RED', 'ORANGE', 'RED', 'BLUE', 'BLUE', 'WHITE', 'BLUE', 'BLUE', 'WHITE', 'WHITE']), ('THURSDAY', ['BLUE', 'BLUE', 'GREEN', 'WHITE', 'BLUE', 'BROWN', 'PINK', 'YELLOW', 'ORANGE', 'CREAM', 'ORANGE', 'RED', 'WHITE', 'BLUE', 'WHITE', 'BLUE', 'BLUE', 'BLUE', 'GREEN']), ('FRIDAY', ['GREEN', 'WHITE', 'GREEN', 'BROWN', 'BLUE', 'BLUE', 'BLACK', 'WHITE', 'ORANGE', 'RED', 'RED', 'RED', 'WHITE', 'BLUE', 'WHITE', 'BLUE', 'BLUE', 'BLUE', 'WHITE'])])

In [12]:
print(list(data.get('TUESDAY'))[0])
print(list(data.get('TUESDAY'))[6])

ARSH
BLEW


# CLEAN-UP OF DATA
It can be seen from the cell above that 2 colors were mispelt. Here, I corrected the mistake.

In [13]:
temp = list(data.get("TUESDAY")) 

for idx, i in enumerate(temp):
    if i == "ARSH":
        temp[idx] = "ASH"
    elif i == "BLEW":
        temp[idx] = "BLUE"
        
data["TUESDAY"] = temp

In [14]:
print(list(data.get('TUESDAY'))[0])
print(list(data.get('TUESDAY'))[6])

ASH
BLUE


In [15]:
all_colors = [j for i in list(data.values()) for j in i] 

In [16]:
print(f"Total colors: {len(all_colors)}")

Total colors: 95


In [17]:
unique_colors = [] 
for i in all_colors:
    if i not in unique_colors:
        unique_colors.append(i) 

In [18]:
print(f"Total unique colors: {len(unique_colors)}")

Total unique colors: 11


## Color and Frequency Dictionary

In [92]:
colors_freq = dict(Counter(sorted(all_colors)))
colors_freq

{'ASH': 1,
 'BLACK': 1,
 'BLUE': 31,
 'BROWN': 6,
 'CREAM': 2,
 'GREEN': 10,
 'ORANGE': 9,
 'PINK': 5,
 'RED': 9,
 'WHITE': 16,
 'YELLOW': 5}

## Q1 solution

**Explanation:** Here, I encoded all colors before calculating the mean.

In [76]:
le = preprocessing.LabelEncoder()
le.fit(all_colors)

code = list(le.transform(all_colors))
color_code = {i:j for i,j in zip(all_colors, code)}

for i, j in zip(color_code.keys(), color_code.values()):
    if j == round(np.mean(code)):
        mean_color = i 

print(f"The mean colour of shirt is {mean_color}")

The mean colour of shirt is GREEN


## Q2 solution

In [22]:
for i in colors_freq.keys():
    if colors_freq.get(i) == max(colors_freq.values()):
        print(f"The most worn colour throughout the week is {i}")

The most worn colour throughout the week is BLUE


## Q3 solution

**Explanation:** Here, I sorted the dictionary containing each colors and their respective frequencies based on the frequency in ascending order before getting the median value 

In [87]:
sorted_colors_freq = dict(sorted(colors_freq.items(), key = itemgetter(1)))
sorted_colors_freq_list = list(sorted_colors_freq.keys())

print(f"The median colour of shirt is {sorted_colors_freq_list[int((len(sorted_colors_freq_list)-1)/2)]}")

The median colour of shirt is BROWN


## Q4 solution

**Explanation:** I used the encoded color values here to get the variance

In [91]:
for i, j in zip(color_code.keys(), color_code.values()):
    if j == round(np.std(code) ** 2):
        variance_color = i 

print(f"The variance of the colors is {variance_color}")

The variance colour of shirt is WHITE


## Q5 solution

In [94]:
print(f"The probability that a randomly choosen color is RED is {round(colors_freq.get('RED')/len(all_colors), 2)}")

The probability that a randomly choosen color is RED is 0.09


## Q6 solution

In [28]:
user_password = "kutu"
host_name = "127.0.0.1"
user_name = "postgres"
port = 5432
cnt = 0
table_name = "color_and_freq"

def create_connection(host_name, user_name, user_password, database = "postgres"):
    connection = None
    
    try:
        connection = psycopg2.connect(host = host_name,
                                   user = user_name,
                                   password = user_password,
                                   port = port,
                                   database = database)
        
        print("Connection to PostgreSQL DB successful", sep = "\n")
    except OperationalError as e:
        print(f"The error '{e}' occurred", sep = "\n")
    
    return connection

def setup():
    connection = create_connection(host_name, user_name, user_password)
    return connection

def execute_query(connection, query):
    global cnt
    
    cnt+=1
    connection.autocommit = True
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
    except OperationalError as e:
        print(f"The error '{e}' occurred")


def execute_read_query(connection, query):
    global cnt
    
    cnt+=1
    cursor = connection.cursor()
    result = None
    
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except OperationalError as e:
        print(f"The error {e} occurred")
        

def color_freq_extractor(colors_freq):
    color = []
    freq = []
    
    for i,j in zip(colors_freq.keys(), colors_freq.values()):
        color.append(i)
        freq.append(j)
    
    return color, freq

def main():
    
    connection = setup()
    
    CREATE_table = f''' 
            CREATE TABLE IF NOT EXISTS {table_name}
            (
             	serial_no SERIAL NOT NULL PRIMARY KEY,
                color TEXT NOT NULL,
                frequency INTEGER NOT NULL
            );
            '''
    
    READ_table = f"SELECT * FROM {table_name};"
    
    execute_query(connection, CREATE_table)
    
    color, freq = color_freq_extractor(colors_freq)
    serial_no = [i for i in range(1, len(color)+1)]
    
    
    for i, j, k in zip(serial_no, color, freq):
        insert_into_table = f'''INSERT INTO {table_name} (serial_no, color, frequency) VALUES 
    {(i, j, k)};'''
        
        execute_query(connection, insert_into_table)
    
    result = execute_read_query(connection, READ_table)
    
    print()
    result_set = pd.DataFrame(result, columns = ["Serial Nos", "Color", "Frequency"])
    print(result_set)
    print()
    
    print(f"Total queries executed successfully: {cnt}")
    
    connection.close()
    
main()

Connection to PostgreSQL DB successful

    Serial Nos   Color  Frequency
0            1     ASH          1
1            2   BLACK          1
2            3    BLUE         31
3            4   BROWN          6
4            5   CREAM          2
5            6   GREEN         10
6            7  ORANGE          9
7            8    PINK          5
8            9     RED          9
9           10   WHITE         16
10          11  YELLOW          5

Total queries executed successfully: 13


## Q7 solution

In [29]:
def recursive_search(item, my_list):
    my_list = sorted(my_list)
    found = False
    first = 0
    last = len(my_list) - 1
    
    while first <= last and found == False:
        midpoint = (first + last) // 2
        
        if my_list[midpoint] == item:
            found = True
        else:
            if my_list[midpoint] < item:
                first = midpoint + 1
            else:
                last = midpoint - 1
    return found


my_list = [70, 24, 87, 45, 6, 3, 2, 8, 5]

user_input = int(input("Enter a number please: "))

status = recursive_search(user_input, my_list)

if status:
    print()
    print(f"Your number {user_input} FOUND in list.")
else:
    print()
    print(f"Your number {user_input} NOT FOUND in list.") 

Enter a number please:  87



Your number 87 FOUND in list.


## Q8 solution

In [34]:
binary_num = "".join(sample(bin(int(digits)), 4))
print(f"{binary_num}\N{Subscript Two} = {int(binary_num, 2)}")

1010₂ = 10


## Q9 solution

In [31]:
terms = 50
fibonacci_seq = [0, 1]

while len(fibonacci_seq) != terms:
    if terms > 2:
        fibonacci_seq.append(sum(fibonacci_seq[-2:]))
        
print(f"The sum of the first {terms} terms of a fibonacci sequence is {sum(fibonacci_seq)}.")
print()

The sum of the first 50 terms of a fibonacci sequence is 20365011073.

